In [1]:
import os
import matplotlib.pyplot
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds

In [2]:
physical_devices = tf.config.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [3]:
(ds_train,ds_test),ds_info =tfds.load(
    "mnist",
    split =["train", "test"],
    shuffle_files = True,
    as_supervised = True,
    with_info = True,

)

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [4]:
"""Normalizes images"""

def normalized_img (iamge,label):
  return tf.cast(iamge,tf.float32) /255.0 , label

In [5]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 128

In [6]:
# Setup for train dataset
ds_train = ds_train.map(normalized_img, num_parallel_calls=AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits["train"].num_examples)
ds_train = ds_train.batch(BATCH_SIZE)
ds_train = ds_train.prefetch(AUTOTUNE)

In [7]:
model = keras.Sequential(
    [
        keras.Input((28, 28, 1)),
        layers.Conv2D(32, 3, activation="relu"),
        layers.Flatten(),
        tf.keras.layers.Dense(10, activation="softmax"),
    ]
)

In [13]:
save_callback = keras.callbacks.ModelCheckpoint(
  '/content' ,
  save_weights_only = True,
  monitor= "train_acc",
  save_best_only = False,
)

In [9]:
lr_scheduler = keras.callbacks.ReduceLROnPlateau(
    monitor="loss", factor=0.1, patience=3, mode="max", verbose=1
)

In [10]:
class OurOwnCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs.get("accuracy") > 1:
            print("Accuracy over 70%, quitting training")
            self.model.stop_training = True

In [11]:
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(),
    optimizer = keras.optimizers.Adam(0.01),
    metrics = ['accuracy']

)

In [14]:
model.fit(
    ds_train,
    epochs=10,
    callbacks=[save_callback, lr_scheduler, OurOwnCallback()],
    verbose=2,
)

Epoch 1/10
469/469 - 8s - loss: 0.1471 - accuracy: 0.9552 - lr: 0.0100 - 8s/epoch - 17ms/step
Epoch 2/10
469/469 - 1s - loss: 0.0573 - accuracy: 0.9819 - lr: 0.0100 - 1s/epoch - 3ms/step
Epoch 3/10
469/469 - 2s - loss: 0.0382 - accuracy: 0.9875 - lr: 0.0100 - 2s/epoch - 3ms/step
Epoch 4/10

Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0009999999776482583.
469/469 - 1s - loss: 0.0266 - accuracy: 0.9913 - lr: 0.0100 - 1s/epoch - 3ms/step
Epoch 5/10
469/469 - 1s - loss: 0.0092 - accuracy: 0.9973 - lr: 1.0000e-03 - 1s/epoch - 3ms/step
Epoch 6/10
469/469 - 1s - loss: 0.0046 - accuracy: 0.9990 - lr: 1.0000e-03 - 1s/epoch - 3ms/step
Epoch 7/10

Epoch 7: ReduceLROnPlateau reducing learning rate to 9.999999310821295e-05.
469/469 - 1s - loss: 0.0029 - accuracy: 0.9996 - lr: 1.0000e-03 - 1s/epoch - 3ms/step
Epoch 8/10
469/469 - 1s - loss: 0.0020 - accuracy: 0.9998 - lr: 1.0000e-04 - 1s/epoch - 3ms/step
Epoch 9/10
469/469 - 1s - loss: 0.0019 - accuracy: 0.9998 - lr: 1.0000e-04 - 1s/epoch